<a href="https://colab.research.google.com/github/CRM48/AIML425-A1/blob/main/Assignmnt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sys
import numpy as np
import plotly.graph_objects as go

# Variables
seed = 1
num_inputs = 5
num_epochs = 1000

train_percent = 0.7
validation_percent = 0.15
test_percent = 0.15

# Check ratio validity
if ((train_percent + validation_percent + test_percent) != 1.0):
    print("Ratio error in data splitting")
    sys.exit(1)

# Generate Data
np.random.seed(seed)
x = np.random.normal(0, 1, (num_inputs, 3))
norms = np.sqrt(np.sum(np.square(x), axis=1, keepdims=True))
y = x / norms

# Split Data
num_samples = x.shape[0]
num_train = int(train_percent * num_samples)
num_validation = int(validation_percent * num_samples)

x_train = x[:num_train]
y_train = y[:num_train]

x_validation = x[num_train:num_train+num_validation]
y_validation = y[num_train:num_train+num_validation]

x_test = x[num_train+num_validation:]
y_test = y[num_train+num_validation:]


print("Input x:")
print(x)

print("Target y (on unit sphere):")
print(y)

print("Target norms")
print(norms)

"""


fig = go.Figure()

# Add normalized y points in blue
fig.add_trace(go.Scatter3d(
    x=y[:, 0], y=y[:, 1], z=y[:, 2],
    mode='markers',
    marker=dict(size=3, color='blue'),
    name='Normalized y'
))

fig.update_layout(
    title="Original Points (red) and Normalized Points on Unit Sphere (blue)",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    )
)

fig.show()
"""

Input x:
[[ 0.05056171  0.49995133 -0.99590893]
 [ 0.69359851 -0.41830152 -1.58457724]
 [-0.64770677  0.59857517  0.33225003]
 [-1.14747663  0.61866969 -0.08798693]
 [ 0.4250724   0.33225315 -1.15681626]]
Target y (on unit sphere):
[[ 0.04532646  0.44818547 -0.89279073]
 [ 0.38975189 -0.23505501 -0.8904171 ]
 [-0.68726067  0.63512873  0.35253975]
 [-0.87821751  0.47349684 -0.06734051]
 [ 0.33301365  0.26029644 -0.90628234]]
Target norms
[[1.11550097]
 [1.77958986]
 [0.94244702]
 [1.3065973 ]
 [1.27644136]]


'\n\n\nfig = go.Figure()\n\n# Add normalized y points in blue\nfig.add_trace(go.Scatter3d(\n    x=y[:, 0], y=y[:, 1], z=y[:, 2],\n    mode=\'markers\',\n    marker=dict(size=3, color=\'blue\'),\n    name=\'Normalized y\'\n))\n\nfig.update_layout(\n    title="Original Points (red) and Normalized Points on Unit Sphere (blue)",\n    scene=dict(\n        xaxis_title=\'X\',\n        yaxis_title=\'Y\',\n        zaxis_title=\'Z\',\n        aspectmode=\'data\'\n    )\n)\n\nfig.show()\n'